In [1]:
!pip install -q chromadb

## Chroma DB 시작

In [2]:
import chromadb

client = chromadb.Client()

collection = client.create_collection("quotes") # quotes라는 이름의 테이블 생성

In [8]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import chromadb  # ChromaDB 라이브러리 사용
import torch

# CSV 파일 읽기
df = pd.read_csv("/home/jdh251425/2025_prometheus/Dataset/quotes_with_insights_combined.csv")

# 모델 로드 (quote_embedding_by_insight.py에서 사용한 모델)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device=device)

# ChromaDB 클라이언트 초기화
client = chromadb.Client()

# 컬렉션이 이미 존재하는지 확인
collection_name = "quotes"
if collection_name not in client.list_collections():
    collection = client.create_collection(collection_name)
else:
    collection = client.get_collection(collection_name)

# 벡터 DB에 데이터 추가
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    # 메타데이터 설정
    metadata = {
        "quote": row['quote'],
        "author": row['author'],
        "category": row['category']
    }
    
    # 임베딩 생성
    embedding = model.encode(row['insight'], convert_to_numpy=True)
    
    # 벡터 DB에 데이터 추가
    collection.add(
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[str(index)]
    )

print("벡터 DB에 데이터 추가 완료!")

100%|██████████| 9996/9996 [02:27<00:00, 67.65it/s]

벡터 DB에 데이터 추가 완료!


## 짧은 글을 벡터화해서 DB에 쿼리로 날려보기

In [15]:
short_text = "오늘 하루는 정말 힘들었다. 업무가 산더미처럼 쌓여 있었고, 해결해야 할 문제들이 끊임없이 나를 괴롭혔다. 하지만 저녁에 여자친구와 통화를 하면서 마음이 한결 가벼워졌다. 그녀의 따뜻한 말 한마디가 큰 위로가 되었고, 다시 힘을 낼 수 있었다. 정말 고맙고, 그녀가 있어서 다행이라는 생각이 든다."

In [16]:
import torch
from sentence_transformers import SentenceTransformer
import chromadb

# 임베딩 모델 로드 (quote_embedding_by_insight.py에서 사용한 모델)
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2', device=device)

# 임베딩할 텍스트
short_text = "오늘 하루는 정말 힘들었다. 업무가 산더미처럼 쌓여 있었고, 해결해야 할 문제들이 끊임없이 나를 괴롭혔다. 하지만 저녁에 여자친구와 통화를 하면서 마음이 한결 가벼워졌다. 그녀의 따뜻한 말 한마디가 큰 위로가 되었고, 다시 힘을 낼 수 있었다. 정말 고맙고, 그녀가 있어서 다행이라는 생각이 든다."

# 텍스트 임베딩
embedding = model.encode(short_text, convert_to_numpy=True)

# ChromaDB 클라이언트 초기화
# client = chromadb.Client()

# 컬렉션 가져오기
collection_name = "quotes"
collection = client.get_collection(collection_name)

# 쿼리 실행
query_result = collection.query(
    query_embeddings=[embedding],
    n_results=3  # 원하는 결과 수
)

# 쿼리 결과 출력
print(query_result)

{'ids': [['4586', '4522', '6075']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'author': 'Maggie Stiefvater, Shiver', 'category': 'giddy, happiness', 'quote': 'right in this moment, I can´t even remember what unhappy feels like.'}, {'author': 'Molly Friedenfeld', 'category': 'inspirational, joy, light, positive-focus, positive-thinking', 'quote': 'We can assist other souls’ hearts and make their world brighter when they are traveling through a difficult experience not by lowering ourselves to their level of sadness but by lifting them up with the light of our joy.'}, {'author': "Mary Karr, The Liars' Club", 'category': 'happiness, metaphor', 'quote': "Just being out of the house with Daddy like this at Fisher's lights me up enough for somebody to read by me."}]], 'distances': [[7.103705883026123, 7.248703956604004, 7.382364273071289]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <Includ

In [26]:
query_result['metadatas'][0]

[{'author': 'Maggie Stiefvater, Shiver',
  'category': 'giddy, happiness',
  'quote': 'right in this moment, I can´t even remember what unhappy feels like.'},
 {'author': 'Molly Friedenfeld',
  'category': 'inspirational, joy, light, positive-focus, positive-thinking',
  'quote': 'We can assist other souls’ hearts and make their world brighter when they are traveling through a difficult experience not by lowering ourselves to their level of sadness but by lifting them up with the light of our joy.'},
 {'author': "Mary Karr, The Liars' Club",
  'category': 'happiness, metaphor',
  'quote': "Just being out of the house with Daddy like this at Fisher's lights me up enough for somebody to read by me."}]